In [1]:
import pandas as pd

In [2]:
guardian_dataset_dir = f"../datasets/Guardian_Dataset.csv"
df = pd.read_csv(guardian_dataset_dir)

In [3]:
row_1_text = df.loc[1, 'Text']
print(row_1_text)

The consensus suggests this will be Antonio Conte’s last Premier League game in charge of Chelsea. Can he use it to somehow secure a top-four finish? Newcastle fans must trust this is not Rafael Benítez’s Tyneside swansong. The negotiations between their widely coveted manager and Mike Ashley over extending a contract that has only a year to run remain delicate. Expect the Gallowgate End to, not for the first time, repeatedly sing “Rafa Benítez, we want you to stay.” Chelsea meanwhile are monitoring Newcastle’s much improved centre-half Jamaal Lascelles. He could be key to Benítez’s team ending a run of four straight defeats as they struggle to retain their grip on 10th place. Louise Taylor


In [4]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Awrod\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Awrod\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [5]:
sentences = sent_tokenize(row_1_text)
for i, sentence in enumerate(sentences):
    print(f"{i+1}): {sentence}")

1): The consensus suggests this will be Antonio Conte’s last Premier League game in charge of Chelsea.
2): Can he use it to somehow secure a top-four finish?
3): Newcastle fans must trust this is not Rafael Benítez’s Tyneside swansong.
4): The negotiations between their widely coveted manager and Mike Ashley over extending a contract that has only a year to run remain delicate.
5): Expect the Gallowgate End to, not for the first time, repeatedly sing “Rafa Benítez, we want you to stay.” Chelsea meanwhile are monitoring Newcastle’s much improved centre-half Jamaal Lascelles.
6): He could be key to Benítez’s team ending a run of four straight defeats as they struggle to retain their grip on 10th place.
7): Louise Taylor


In [9]:
from transformers import pipeline

# model = pipeline("text-generation", model="EleutherAI/gpt-neo-2.7B")
model = pipeline("text-generation", model="EleutherAI/gpt-neo-2.7B", device=0)

RuntimeError: [enforce fail at ..\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 104857600 bytes.


test: The consensus suggests this will be Antonio Conte’s last Premier League game in charge of Chelsea.



In [8]:
home_team = df.loc[1, 'Home']
away_team = df.loc[1, 'Away']

prompt_text = f"""
You will be given a setence that is extracted from a preview of a football match.
The home team is {home_team} and the away team is {away_team}.
Classify which team the sentence is talking about.
All you must do is respond 'home', 'away', 'both', or 'neither'.
The sentence you need to classify is: {sentences[0]}
"""

response = model((prompt_text), max_length=150, num_return_sequences=1)
print(response[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



You will be given a setence that is extracted from a preview of a football match.
The home team is Newcastle United and the away team is Chelsea.
Classify which team the sentence is talking about.
All you must do is respond 'home', 'away', 'both', or 'neither'.
The sentence you need to classify is: The consensus suggests this will be Antonio Conte’s last Premier League game in charge of Chelsea.
You are asked to pick either one out of four possible outcomes.
In all cases, you need to pick three out of four.
The correct response is: 'Both'.
The sentence you must classify is:
Both Manchester United have endured a difficult first half


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium", padding_side='left')
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

for step in range(5):
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/roberta-base-squad2"
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)

QA_input = {
    'question': 'Why is model conversion important?',
    'context': 'The option to convert models between FARM and transformers gives freedom to the user and let people easily switch between frameworks.'
}
print(nlp(QA_input))

QA_input = {
    'question': 'Why is a good programming language?',
    'context': 'For robotics.'
}
print(nlp(QA_input))

In [ ]:
from transformers import pipeline

distilled_student_sentiment_classifier = pipeline(
    model="lxyuan/distilbert-base-multilingual-cased-sentiments-student", 
    return_all_scores=True
)

distilled_student_sentiment_classifier("This team is obviously going to lose.")
distilled_student_sentiment_classifier("There have been a change in management.")

In [ ]:
from setfit import SetFitModel, Trainer
from datasets import Dataset

# Define your few-shot samples
data = {
    "text": [
        "The players are tired from the previous game",
        "There is a change in management",
        "The players are cheerful from winning a close one last week"
    ],
    "label": [
        "negative",  # Class for first sample
        "negative",  # Class for second sample
        "Positive"   # Class for third sample
    ]
}

# Create a dataset from the few-shot samples
dataset = Dataset.from_dict(data)

# Map text labels to integers (required for training)
label_mapping = {"Positive": 0, "negative": 1, "neutral": 2}
dataset = dataset.map(lambda x: {"label": label_mapping[x["label"]]})

# Load a pretrained SetFit model
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")

# Train the model using the few-shot dataset
trainer = Trainer(
    model=model,
    train_dataset=dataset,
    metric="accuracy"
)

trainer.train()

# Use the trained model for inference
predictions = model(["The team is hopeful about their upcoming match"])
predicted_label = [list(label_mapping.keys())[label] for label in predictions]

print(predicted_label)
